# The image slurper
Are you **tired** of asking people for the data behind their published false color plots? Use the image slurper to find the underlying matrix of values from a false-color (heat map) image and its colorbar! 

The slurper can be used on many types of false color plots, and works best on relatively smooth data.

In [ ]:
%reset -f
%run imageslurper.py

As an example, here is a typical pseudoprojection plot of NASA brightness temperature data and a corresponding colorbar. This plot is not ideal as the ocean appears to be uniformly set to the lowest value which generates sharp gradients along coastlines.

![title](img/world-temp.jpg)

Next, run the following cell and use the mouse to draw boxes around the plot area and around the colorbar area in the cell output:

In [ ]:
file = "img/world-temp.jpg"
full_image = PIL.Image.open(file)

%matplotlib notebook

fig, current_ax = plt.subplots(figsize=(10,4))
current_ax.imshow(full_image)
current_ax.axis('off')

boxes = [Rectangle((1, 1), 0, 0, facecolor='green', edgecolor='green', alpha=0.4),
         Rectangle((1, 1), 0, 0, facecolor='blue', edgecolor='blue', alpha=0.4)]

current_ax.add_patch(boxes[0])
current_ax.add_patch(boxes[1])


count=0
numbox=len(boxes)
def line_select_callback(eclick, erelease):
    global count
    'eclick and erelease are the press and release events'
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata

    bid = count % numbox
    boxes[bid].set_x(x1)
    boxes[bid].set_y(y1)
    boxes[bid].set_width(x2-x1)
    boxes[bid].set_height(y2-y1)
    
    count += 1

RS1 = RectangleSelector(current_ax, line_select_callback,
                                       drawtype='box', useblit=True,
                                       button=[1],  # don't use middle button
                                       minspanx=5, minspany=5,
                                       spancoords='pixels',
                                       interactive=True,
                                       rectprops=dict(facecolor='none', edgecolor = 'black',
                                                      alpha=1, fill=True))

plt.show()

If tick marks extend into the colorbar, try to avoid that region. The mouse interactions loosely crop out the map area  and the colorbar by specifying `map_corners` and `bar_corners`. The two areas can overlap but the map area must not contain any part of the colorbar, or vice versa. The values can also be given by hand below.

In [ ]:
%matplotlib inline

map_corners = [(m.x0, m.y0, m.x1, m.y1) for m in (boxes[0].get_bbox(),)][0]
bar_corners = [(m.x0, m.y0, m.x1, m.y1) for m in (boxes[1].get_bbox(),)][0]

print("Map corners: ", map_corners)
print("Colorbar corners:", bar_corners)

The notebook should now isolate the map area and the colorbar from the rest of the image. The reconstruction uses a horizontal colorbar, accomplished by the `rotate` method if necessary. The lowest value should be on the left. 

In [ ]:
map_image = full_image.crop(map_corners)
colorbar_img = full_image.crop(bar_corners).rotate(0, expand=True)

map_image = crop(map_image, thresh=80)
colorbar_img = crop(colorbar_img, thresh=100)

display('Map', map_image, 'Colorbar', colorbar_img)

Resize the image to make the processing faster (optional).

In [ ]:
new_dimensions = 600, 300
image = map_image.resize(new_dimensions, PIL.Image.LANCZOS)
rgb_img = np.array(image)

assert not np.any(np.isnan(rgb_img)), "No NaN values allowed."

fig, axs = plt.subplots(1,2, figsize=(12,8))
axs[0].imshow(rgb_img)
axs[0].set_title("Resized image array shape " + str(rgb_img.shape))

colorbar_img = np.array(colorbar_img)
colorbar_rgb = np.average(colorbar_img, axis=0)

assert not np.any(np.isnan(colorbar_img)), "No NaN values allowed."

ax = axs[1]
ax.imshow(colorbar_img, extent=(0, colorbar_img.shape[1], 0, 256))
for _id, color in enumerate('rgb'):
    ax.plot(colorbar_rgb[..., _id], color=color)
ax.invert_yaxis()
ax.set_title('Colorbar RGB intensities')
ax.set_ylim(0, 256)
ax.set_yticks(np.linspace(0, 256, 9))
plt.legend('RGB')
plt.show()

The reconstruction alogrithm uses brute force and picks the nearest color in the colormap for each pixel and assigns a value in the range $[0,1]$. The error is also in the range $[0,1]$.

In [ ]:
# See https://stackoverflow.com/questions/43843381/digitize-a-colormap
def unmap_nearest(img, rgb):
    """ img is an image of shape [n, m, 3], and rgb is a colormap of shape [k, 3]. """
    d = np.sum(np.abs(img[np.newaxis, ...] - rgb[:, np.newaxis, np.newaxis, :]), axis=-1)    
    i = np.argmin(d, axis=0)
    err = np.min(d, axis=0)
    return i / (rgb.shape[0] - 1), err / (255*3)

reconstructed, err = unmap_nearest(rgb_img, colorbar_rgb)

Scale the image to the expected by setting `vmin` and `vmax` to the values indicated on the colorbar. 

In [ ]:
vmin, vmax = 180, 280
reconstructed = reconstructed * (vmax - vmin) + vmin
err = err * (vmax - vmin)

Create arrays of $x,y$ values to match the original images axes

In [ ]:
x = np.linspace(-180, 180, reconstructed.shape[1])
y = np.linspace(90, -90, reconstructed.shape[0])
X, Y = np.meshgrid(x,y)

assert not np.any(np.isnan(reconstructed)), "No NaN values allowed."
assert not np.any(np.isnan(err)), "No NaN values allowed."

fig, ax = plt.subplots(figsize=(14,6))
img = ax.pcolormesh(X, Y, reconstructed, cmap='viridis')
ax.set_title('Reconstructed dataset')
plt.colorbar(img,ax=ax)

fig, ax = plt.subplots(figsize=(14,6))
ax.set_title('Reconstruction error')
img = ax.pcolormesh(X, Y, err, cmap='magma')
plt.colorbar(img,ax=ax)

Remove the tick marks; in areas where the error is larger than `thresh` replace the value by the median of its neighbours. Set the threshold value as low as possible to capture the tick marks extending into the plot area, but not the interior of the plot.

In [ ]:
error_threshold=15

fig, ax = plt.subplots(1, 1, figsize=(14,6))
ax.set_title('Bad pixels')
img = ax.imshow(1.0 * (err > error_threshold), cmap='magma')
fig.colorbar(img, ax=ax)

In [ ]:
def hole_fill(data, error, di=5, thresh=.02):

    data_padded = np.empty([i+2*di for i in data.shape])
    data_padded.fill(np.nan)

    data_padded[di:-di, di:-di] = reconstructed
    data_padded[di:-di, di:-di][error>thresh]=np.nan

    result = data
    ids = np.where(error>thresh)
    ids = np.array(ids)
    for x_id, y_id in ids.T:
        x_id+=di
        y_id+=di

        result[x_id-di, y_id-di] = np.nanmedian(data_padded[x_id-di:x_id+di, y_id-di:y_id+di])

    return result

filled = hole_fill(reconstructed, err, thresh=error_threshold)

Plot the reconstructed map with the original colormap. The images should be very similar.

In [ ]:
IPython.display.display(full_image)

if np.any(np.isnan(filled)):
    print("Some NaN values could not be filled.")

fig, ax = plt.subplots(figsize=(14,6))
original_colormap = ListedColormap(colorbar_rgb/255)
img = ax.pcolormesh(X, Y, filled, cmap=original_colormap, vmin=vmin, vmax=vmax)
print("Image values range: [%f, %f]" % (filled.min(), filled.max()))

fig.colorbar(img)
plt.title('Reconstructed dataset using original colorbar')
plt.show()

Save the image and the colorbar RGB values in a `pickle` file.

In [ ]:
save_file = basename(file) + "-slurped.p"
with open(save_file, 'wb') as save_handle:
    pickle.dump([filled, colorbar_rgb],
                save_handle,
                protocol=pickle.HIGHEST_PROTOCOL)

# This reads the objects back:
with open(save_file, 'rb') as load_handle:
    filled, colorbar_rgb = pickle.load(load_handle)

Or save the image as a `csv` file

In [ ]:
np.savetxt(basename(file) + "-slurped.csv", 
              filled, 
              delimiter=", ",
              fmt="%0.6e",
              header=make_header(file, filled.shape))